In [1]:
%%capture
!pip3 install --no-cache-dir --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import polara
from polara import get_movielens_data
from polara.preprocessing.dataframes import leave_one_out, reindex

from dataprep import transform_indices
from evaluation import topn_recommendations, model_evaluate, downvote_seen_items

from polara.lib.tensor import hooi
from polara.lib.sparse import tensor_outer_at

from sa_hooi import sa_hooi, form_attention_matrix, get_scaling_weights, generate_position_projector

from polara.lib.sparse import tensor_outer_at
from scipy.sparse import csr_matrix
from scipy.linalg import solve_triangular

%load_ext autoreload
%autoreload 2

In [2]:
def full_preproccessing():
    data = get_movielens_data(include_time=True)
    test_timepoint = data['timestamp'].quantile(
    q=0.9, interpolation='nearest'
    )

    test_data_ = data.query('timestamp >= @test_timepoint')
    train_data_ = data.query(
    'userid not in @test_data_.userid.unique() and timestamp < @test_timepoint'
    )
    
    training, data_index = transform_indices(train_data_.copy(), 'userid', 'movieid')
    test_data = reindex(test_data_, data_index['items'])

    testset_, holdout_ = leave_one_out(
    test_data, target='timestamp', sample_top=True, random_state=0
    )
    testset_valid_, holdout_valid_ = leave_one_out(
        testset_, target='timestamp', sample_top=True, random_state=0
    )

    test_users_val = np.intersect1d(testset_valid_.userid.unique(), holdout_valid_.userid.unique())
    testset_valid = testset_valid_.query('userid in @test_users_val').sort_values('userid')
    holdout_valid = holdout_valid_.query('userid in @test_users_val').sort_values('userid')

    test_users = np.intersect1d(testset_.userid.unique(), holdout_.userid.unique())
    testset = testset_.query('userid in @test_users').sort_values('userid')
    holdout = holdout_.query('userid in @test_users').sort_values('userid')


    assert holdout_valid.set_index('userid')['timestamp'].ge(
        testset_valid
        .groupby('userid')
        ['timestamp'].max()
    ).all()

    data_description = dict(
        users = data_index['users'].name,
        items = data_index['items'].name,
        feedback = 'rating',
        n_users = len(data_index['users']),
        n_items = len(data_index['items']),
        n_ratings = training['rating'].nunique(),
        min_rating = training['rating'].min()
    )

    return training, testset_valid, holdout_valid, testset, holdout, data_description

In [3]:
training, testset_valid, holdout_valid, testset, holdout, data_description = full_preproccessing()

Filtered 113 invalid observations.


In [4]:
from IPython.utils import io

def tf_model_build(config, data, data_description, attention_matrix=np.array([])):
    userid = data_description["users"]
    itemid = data_description["items"]
    feedback = data_description["feedback"]

    idx = data[[userid, itemid, feedback]].values
    idx[:, -1] = idx[:, -1] - data_description['min_rating'] # works only for integer ratings!
    val = np.ones(idx.shape[0], dtype='f8')
    
    n_users = data_description["n_users"]
    n_items = data_description["n_items"]
    n_ratings = data_description["n_ratings"]
    shape = (n_users, n_items, n_ratings)
    core_shape = config['mlrank']
    num_iters = config["num_iters"]
    
    if (attention_matrix.shape[0] == 0):
        attention_matrix = form_attention_matrix(
            data_description['n_ratings'],
            **config['params'],
            format = 'csr'
        )

    item_popularity = (
        data[itemid]
        .value_counts(sort=False)
        .reindex(range(n_items))
        .fillna(1)
        .values
    )
    scaling_weights = get_scaling_weights(item_popularity, scaling=config["scaling"])

    with io.capture_output() as captured:
        u0, u1, u2 = sa_hooi(
            idx, val, shape, config["mlrank"],
            attention_matrix = attention_matrix,
            scaling_weights = scaling_weights,
            max_iters = config["num_iters"],
            parallel_ttm = False,
            randomized = config["randomized"],
            growth_tol = config["growth_tol"],
            seed = config["seed"],
            iter_callback = None,
        )
    
    return u0, u1, u2, attention_matrix    
    
config = {
    "scaling": 1,
    "mlrank": (30, 30, 5),
    "n_ratings": data_description['n_ratings'],
    "num_iters": 3,
    "params": None,
    "randomized": True,
    "growth_tol": 1e-4,
    "seed": 42
}


def tf_scoring(params, data, data_description, context=["3+4+5"]):
    user_factors, item_factors, feedback_factors, attention_matrix = params
    userid = data_description["users"]
    itemid = data_description["items"]
    feedback = data_description["feedback"]

    data = data.sort_values(userid)
    useridx = data[userid]
    itemidx = data[itemid].values
    ratings = data[feedback].values
    ratings = ratings - data_description['min_rating']
    
    n_users = useridx.nunique()
    n_items = data_description['n_items']
    n_ratings = data_description['n_ratings']
    
    #inv_attention = np.linalg.inv(attention_matrix.A) # change
    inv_attention = solve_triangular(attention_matrix, np.eye(5), lower=True)
    #np.testing.assert_almost_equal(inv_attention, inv_attention_)
    
    tensor_outer = tensor_outer_at('cpu')
    
    if (context == "5"):
        inv_aT_feedback = (inv_attention.T @ feedback_factors)[-1, :]
    elif (context == "4+5"):
        inv_aT_feedback = np.sum((inv_attention.T @ feedback_factors)[-2:, :], axis=0)
    elif (context == "3+4+5"):
        inv_aT_feedback = np.sum((inv_attention.T @ feedback_factors)[-3:, :], axis=0)
    elif (context == "2+3+4+5"):
        inv_aT_feedback = np.sum((inv_attention.T @ feedback_factors)[-4:, :], axis=0)
    elif (context == "3+4+5-2-1"):
        inv_aT_feedback = np.sum((inv_attention.T @ feedback_factors)[-3:, :], axis=0) - np.sum((inv_attention.T @ feedback_factors)[:2, :], axis=0)
        
    scores = tensor_outer(
        1.0,
        item_factors,
        attention_matrix @ feedback_factors,
        itemidx,
        ratings
    )
    scores = np.add.reduceat(scores, np.r_[0, np.where(np.diff(useridx))[0]+1]) # sort by users
    scores = np.tensordot(
        scores,
        inv_aT_feedback,
        axes=(2, 0)
    ).dot(item_factors.T)

#     scores = np.zeros((n_users, n_items))
#     print(scores.shape)
#     #inv_attention = np.linalg.inv(attention_matrix.A)
#     for i, u in tqdm(enumerate(np.unique(useridx))):
#         data_u = data[data.userid==u]
#         P = csr_matrix((np.ones(data_u.shape[0]), (data_u[itemid].values, data_u[feedback].values - data_description['min_rating'])), (n_items, n_ratings))
#         res = item_factors @ (item_factors.T @ (P @ (attention_matrix @ (feedback_factors @ (inv_attention.T @ feedback_factors).T))))
#         if (context == "5"):
#             scores[i] = np.sum(res[:, -1:], axis=1)
#         elif (context == "4+5"):
#             scores[i] = np.sum(res[:, -2:], axis=1)
#         elif (context == "3+4+5"):
#             scores[i] = np.sum(res[:, -3:], axis=1)
#         elif (context == "2+3+4+5"):
#             scores[i] = np.sum(res[:, -4:], axis=1)
#         elif (context == "3+4+5-2-1"):
#             scores[i] = np.sum(res[:, 2:], axis=1) - np.sum(res[:, :2], axis=1)
        
#     if (context == "4+5"):
#         np.testing.assert_almost_equal(scores, scores_)
    return scores

def model_evaluate(recommended_items, holdout, holdout_description, alpha=3, topn=10):
    itemid = holdout_description['items']
    rateid = holdout_description['feedback']
    holdout_items = holdout[itemid].values
    assert recommended_items.shape[0] == len(holdout_items)
    hits_mask = recommended_items[:, :topn] == holdout_items.reshape(-1, 1)
    # HR calculation
    hr = np.mean(hits_mask.any(axis=1))
    # MRR calculation
    n_test_users = recommended_items.shape[0]
    hit_rank = np.where(hits_mask)[1] + 1.0
    mrr = np.sum(1 / hit_rank) / n_test_users
    # DCG calculation
    pos_mask = (holdout[rateid] >= alpha).values
    neg_mask = (holdout[rateid] < alpha).values
    pos_hit_rank = np.where(hits_mask[pos_mask])[1] + 1.0
    neg_hit_rank = np.where(hits_mask[neg_mask])[1] + 1.0
    ndcg = np.sum(1 / np.log2(pos_hit_rank+1)) / n_test_users
    ndcl = np.sum(1 / np.log2(neg_hit_rank+1)) / n_test_users
    # coverage calculation
    n_items = holdout_description['n_items']
    cov = np.unique(recommended_items).size / n_items
    return hr, mrr, cov, ndcg, ndcl

def make_prediction(tf_scores, holdout, data_description, mode, context="", print_mode=True):
    if (mode and print_mode):
        print(f"for context {context} evaluation:")
    for n in [5, 10, 20]:
        tf_recs = topn_recommendations(tf_scores, n)
        hr, mrr, cov, dcg, dcl = model_evaluate(tf_recs, holdout, data_description, topn=n)
        if (print_mode):
            print(f"Test : HR@{n} = {hr:.4f}, MRR@{n} = {mrr:.4f}, Coverage@{n} = {cov:.4f}, nDCG@{n} = {dcg:.5f}, nDCL@{n} = {dcl:.5f}")
        if (n == 10):
            mrr10 = mrr
            hr10 = hr
    return mrr10, hr10

def valid_mlrank(mlrank):
    '''
    Only allow ranks that are suitable for truncated SVD computations
    on unfolded compressed tensor (the result of ttm product in HOOI).
    '''
    #s, r1, r2, r3 = mlrank
    s, r1, r3 = mlrank
    r2 = r1
    #print(s, r1, r2, r3)
    return r1*r2 > r3 and r1*r3 > r2 and r2*r3 > r1

In [5]:
def best_num_iter_sa_hooi(config, training, data_description, attention_matrix, testset, holdout):
    # Claculate optimal number of iteration for each context for metrics : MRR@10, HR@10
    # Input:
    # config - dict of params
    # training - training dataset
    # data_description - dict of data_description
    # attention_matrix - attention_matrix which will be used in sa_hooi algo
    # testset - test or validation part of dataset
    # holdout - holdout which corresponding to users from testset
    
    print("Starting tuning number of iteration in sa_hooi algo...\n")
    init_max_iter = config["num_iters"]
    for context in ["5", "4+5", "3+4+5", "2+3+4+5", "3+4+5-2-1"]:
        best_mrr, best_hr = -1, -1
        for max_iter in range(1, 10):
            config["num_iters"] = max_iter
            tf_params = tf_model_build(config, training, data_description, attention_matrix=attention_matrix)
            tf_scores = tf_scoring(tf_params, testset, data_description, context)
            downvote_seen_items(tf_scores, testset, data_description)
            cur_mrr, cur_hr = make_prediction(tf_scores, holdout, data_description, "Validation", context, print_mode=False)
            if (cur_mrr > best_mrr):
                best_mrr = cur_mrr
                best_iter_mrr = max_iter
            if (cur_hr > best_hr):
                best_hr = cur_hr
                best_iter_hr = max_iter
            
        print(f"For context '{context}' the following parameters were found:")
        print(f"\tFor HR@10 ({best_hr:.4f}) -- number of iteration is {best_iter_hr}")
        print(f"\tFor MRR@10 ({best_mrr:.4f}) -- number of iteration is {best_iter_mrr}")
        print("------------------------------------------------------\n")
    config["num_iters"] = init_max_iter
    print("Tuning ended.")
    return 


In [6]:
from tqdm import tqdm 
from polara.evaluation.pipelines import random_grid

def full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix):

    config["mlrank"] = (30, 30, 5)
    print("Starting pipeline...")
    print("Training with different context in progress...")
    print("------------------------------------------------------")
    best_mrr_context = "3+4+5"
    best_mrr = 0.0
    for context in ["5", "4+5", "3+4+5", "2+3+4+5", "3+4+5-2-1"]:
        tf_params = tf_model_build(config, training, data_description, attention_matrix=attention_matrix)
        seen_data = testset_valid
        tf_scores = tf_scoring(tf_params, seen_data, data_description, context)
        downvote_seen_items(tf_scores, seen_data, data_description)
        cur_mrr, cur_hr = make_prediction(tf_scores, holdout_valid, data_description, "Validation", context)
        print("------------------------------------------------------")
        if (cur_mrr > best_mrr):
            best_mrr = cur_mrr
            best_mrr_context = context

    best_mrr_context = "3+4+5" # intuitively this is better 
    #print(f"Tuning model with context {best_mrr_context}...")
    print(f"Tuning model for all contexts...\n")

    tf_hyper = {
    'scaling': np.array([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]),
    'r1': np.arange(100, 301, 25),
    #'r2': np.arange(50, 801, 25),
    'r3': range(2, 6, 1),
    }

    grid, param_names = random_grid(tf_hyper, n=0)
    tf_grid = [tuple(mlrank) for mlrank in grid if valid_mlrank(mlrank)]

    hr_tf = {}
    mrr_tf = {}
    cov_tf = {}
    seen_data = testset_valid
    full_result = dict()
    for mlrank in tqdm(tf_grid):
        with io.capture_output() as captured:
            r1, r3 = mlrank[1:]
            cur_mlrank = tuple((r1, r1, r3))
            config['mlrank'] = cur_mlrank
            config['scaling'] = mlrank[0]
            tf_params = tf_model_build(config, training, data_description, attention_matrix=attention_matrix)
            for context in ["5", "4+5", "3+4+5", "2+3+4+5", "3+4+5-2-1"]:
                tf_scores = tf_scoring(tf_params, seen_data, data_description, context)
                downvote_seen_items(tf_scores, seen_data, data_description)
                tf_recs = topn_recommendations(tf_scores, topn=10)
                hr, mrr, cov, dcg, dcl = model_evaluate(tf_recs, holdout_valid, data_description, topn=10)
                hr_tf[(context, cur_mlrank, mlrank[0])] = hr
                mrr_tf[(context, cur_mlrank, mlrank[0])] = mrr
                cov_tf[(context, cur_mlrank, mlrank[0])] = cov

    print(f'Best HR={pd.Series(hr_tf).max():.4f} achieved with context {pd.Series(hr_tf).idxmax()[0]} and mlrank = {pd.Series(hr_tf).idxmax()[1]} and scale factor = {pd.Series(hr_tf).idxmax()[2]}')
    print(f'Best MRR={pd.Series(mrr_tf).max():.4f} achieved with context {pd.Series(mrr_tf).idxmax()[0]} and mlrank = {pd.Series(mrr_tf).idxmax()[1]} and scale factor = {pd.Series(mrr_tf).idxmax()[2]}')
    print(f'COV={pd.Series(cov_tf)[pd.Series(mrr_tf).idxmax()]:.4f} (based on best MRR value)')
    print("---------------------------------------------------------")
    print("Evaluation of the best model on test holdout in progress...\n")
    
    print("Best by MRR@10:\n")
    config["mlrank"] = pd.Series(mrr_tf).idxmax()[1]
    tf_params = tf_model_build(config, training, data_description, attention_matrix=attention_matrix)

    seen_data = testset
    tf_scores = tf_scoring(tf_params, seen_data, data_description, pd.Series(mrr_tf).idxmax()[0])
    downvote_seen_items(tf_scores, seen_data, data_description)
    cur_mrr, cur_hr = make_prediction(tf_scores, holdout, data_description, "Test", pd.Series(mrr_tf).idxmax()[0])
    
    print("---------------------------------------------------------")
    
    print("Best by HR@10:\n")
    config["mlrank"] = pd.Series(hr_tf).idxmax()[1]
    tf_params = tf_model_build(config, training, data_description, attention_matrix=attention_matrix)

    seen_data = testset
    tf_scores = tf_scoring(tf_params, seen_data, data_description, pd.Series(hr_tf).idxmax()[0])
    downvote_seen_items(tf_scores, seen_data, data_description)
    cur_mrr, cur_hr = make_prediction(tf_scores, holdout, data_description, "Test", pd.Series(hr_tf).idxmax()[0])
    print("Pipeline ended.")

## Linear attention

In [17]:
linear_attentions_list = [
    {'decay_factor': 1, 'exponential_decay': False, 'reverse': False},
    #{'decay_factor': 1, 'exponential_decay': False, 'reverse': False},
]

In [18]:
for params in linear_attentions_list:
    
    config["params"] = params
    
    #best_num_iter_sa_hooi(config, training, data_description, np.array([]), testset_valid, holdout_valid)
    full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix=np.array([]))

Starting pipeline...
Training with different context in progress...
------------------------------------------------------
for context 5 evaluation:
Test : HR@5 = 0.0255, MRR@5 = 0.0142, Coverage@5 = 0.1133, nDCG@5 = 0.01701, nDCL@5 = 0.00000
Test : HR@10 = 0.0440, MRR@10 = 0.0168, Coverage@10 = 0.1618, nDCG@10 = 0.02248, nDCL@10 = 0.00063
Test : HR@20 = 0.0792, MRR@20 = 0.0193, Coverage@20 = 0.2239, nDCG@20 = 0.03072, nDCL@20 = 0.00132
------------------------------------------------------
for context 4+5 evaluation:
Test : HR@5 = 0.0317, MRR@5 = 0.0171, Coverage@5 = 0.1095, nDCG@5 = 0.02032, nDCL@5 = 0.00034
Test : HR@10 = 0.0651, MRR@10 = 0.0213, Coverage@10 = 0.1508, nDCG@10 = 0.03001, nDCL@10 = 0.00119
Test : HR@20 = 0.1082, MRR@20 = 0.0242, Coverage@20 = 0.2080, nDCG@20 = 0.03949, nDCL@20 = 0.00249
------------------------------------------------------
for context 3+4+5 evaluation:
Test : HR@5 = 0.0352, MRR@5 = 0.0185, Coverage@5 = 0.1158, nDCG@5 = 0.02183, nDCL@5 = 0.00072
Test 

  0%|          | 0/216 [00:00<?, ?it/s]

for context 3+4+5-2-1 evaluation:
Test : HR@5 = 0.0308, MRR@5 = 0.0173, Coverage@5 = 0.1164, nDCG@5 = 0.02002, nDCL@5 = 0.00055
Test : HR@10 = 0.0580, MRR@10 = 0.0208, Coverage@10 = 0.1546, nDCG@10 = 0.02792, nDCL@10 = 0.00133
Test : HR@20 = 0.1055, MRR@20 = 0.0240, Coverage@20 = 0.2099, nDCG@20 = 0.03943, nDCL@20 = 0.00173
------------------------------------------------------
Tuning model for all contexts...



100%|██████████| 216/216 [1:59:13<00:00, 33.12s/it]  


Best HR=0.1038 achieved with context 2+3+4+5 and mlrank = (275, 275, 5) and scale factor = 0.5
Best MRR=0.0428 achieved with context 2+3+4+5 and mlrank = (300, 300, 4) and scale factor = 0.5
COV=0.3010 (based on best MRR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...

Best by MRR@10:

for context 2+3+4+5 evaluation:
Test : HR@5 = 0.0519, MRR@5 = 0.0263, Coverage@5 = 0.2355, nDCG@5 = 0.02570, nDCL@5 = 0.00685
Test : HR@10 = 0.0739, MRR@10 = 0.0292, Coverage@10 = 0.2996, nDCG@10 = 0.03199, nDCL@10 = 0.00766
Test : HR@20 = 0.1126, MRR@20 = 0.0318, Coverage@20 = 0.3777, nDCG@20 = 0.04073, nDCL@20 = 0.00861
---------------------------------------------------------
Best by HR@10:

for context 2+3+4+5 evaluation:
Test : HR@5 = 0.0475, MRR@5 = 0.0239, Coverage@5 = 0.2264, nDCG@5 = 0.02358, nDCL@5 = 0.00611
Test : HR@10 = 0.0721, MRR@10 = 0.0272, Coverage@10 = 0.2927, nDCG@10 = 0.03067, nDCL@10 = 0.00700
Test : HR@20 

# Custom attention

In [7]:
similarity_matrix = np.array([[1, 0.85, 0.5, 0.1, 0], [0.85, 1, 0.65, 0.2, 0.05], [0.5, 0.65, 1, 0.75, 0.2], [0.1, 0.2, 0.75, 1, 0.85], [0, 0.05, 0.2, 0.85, 1]])
similarity_matrix

array([[1.  , 0.85, 0.5 , 0.1 , 0.  ],
       [0.85, 1.  , 0.65, 0.2 , 0.05],
       [0.5 , 0.65, 1.  , 0.75, 0.2 ],
       [0.1 , 0.2 , 0.75, 1.  , 0.85],
       [0.  , 0.05, 0.2 , 0.85, 1.  ]])

In [8]:
import scipy
attention_matrix = scipy.linalg.sqrtm(similarity_matrix).real
attention_matrix

array([[ 0.86030365,  0.46408882,  0.21104287, -0.0075498 , -0.00117653],
       [ 0.46408882,  0.8228861 ,  0.32961986,  0.02604066,  0.02409949],
       [ 0.21104287,  0.32961986,  0.79451066,  0.48479688, -0.0301396 ],
       [-0.0075498 ,  0.02604066,  0.48479688,  0.71597228,  0.53580926],
       [-0.00117653,  0.02409949, -0.0301396 ,  0.53580926,  0.85263566]])

In [ ]:
config["params"] = {}

full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix=attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------
for context 5 evaluation:
Test : HR@5 = 0.0299, MRR@5 = 0.0161, Coverage@5 = 0.1062, nDCG@5 = 0.01947, nDCL@5 = 0.00000
Test : HR@10 = 0.0624, MRR@10 = 0.0203, Coverage@10 = 0.1475, nDCG@10 = 0.02957, nDCL@10 = 0.00029
Test : HR@20 = 0.0994, MRR@20 = 0.0228, Coverage@20 = 0.2039, nDCG@20 = 0.03734, nDCL@20 = 0.00183
------------------------------------------------------
for context 4+5 evaluation:
Test : HR@5 = 0.0299, MRR@5 = 0.0175, Coverage@5 = 0.1078, nDCG@5 = 0.02008, nDCL@5 = 0.00044
Test : HR@10 = 0.0668, MRR@10 = 0.0223, Coverage@10 = 0.1486, nDCG@10 = 0.03109, nDCL@10 = 0.00126
Test : HR@20 = 0.1196, MRR@20 = 0.0260, Coverage@20 = 0.1997, nDCG@20 = 0.04317, nDCL@20 = 0.00261
------------------------------------------------------
for context 3+4+5 evaluation:
Test : HR@5 = 0.0299, MRR@5 = 0.0169, Coverage@5 = 0.1125, nDCG@5 = 0.01963, nDCL@5 = 0.00044
Test 

  0%|          | 1/324 [02:20<12:35:38, 140.37s/it]

## Exponential attention, decay factor = 1

In [ ]:
exponential_attentions_list = [
    {'decay_factor': 1, 'exponential_decay': True, 'reverse': True},
    {'decay_factor': 1, 'exponential_decay': True, 'reverse': False}
]

In [ ]:
for params in exponential_attentions_list:
    
    config["params"] = params
    
    full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix=np.array([]))

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1346.81it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0299, MRR@5 == 0.0162, Coverage@5 = 0.0754
Validation : HR@10 = 0.0431, MRR@10 == 0.0180, Coverage@10 = 0.1111
Validation : HR@20 = 0.0853, MRR@20 == 0.0207, Coverage@20 = 0.1565
------------------------------------------------------


1137it [00:00, 1391.04it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0290, MRR@5 == 0.0133, Coverage@5 = 0.0779
Validation : HR@10 = 0.0545, MRR@10 == 0.0166, Coverage@10 = 0.1092
Validation : HR@20 = 0.0897, MRR@20 == 0.0188, Coverage@20 = 0.1538
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0317, MRR@5 == 0.0163, Coverage@5 = 0.1073
Validation : HR@10 = 0.0519, MRR@10 == 0.0189, Coverage@10 = 0.1469
Validation : HR@20 = 0.0932, MRR@20 == 0.0218, Coverage@20 = 0.2072
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [04:02<00:00,  6.74s/it]


Best HR=0.0712 achieved with mlrank=(55, 45, 5)
Best MRR=0.0260 achieved with mlrank=(55, 45, 5)
COV=0.1678 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1129.78it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0229, MRR@5 == 0.0121, Coverage@5 = 0.1241
Test : HR@10 = 0.0493, MRR@10 == 0.0157, Coverage@10 = 0.1673
Test : HR@20 = 0.1003, MRR@20 == 0.0191, Coverage@20 = 0.2261
Pipeline ended.


## Eucledian distance attention

In [ ]:
eucl_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        eucl_matrix[i, j] = 1.0 / np.exp(abs(i - j)) if i != j else 1#5 + 1e-2
        
a = np.linalg.cholesky(eucl_matrix)

#for i in range(5):
#    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)

In [ ]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:02, 454.89it/s]


for context 5 evaluation:
Test : HR@5 = 0.0281, MRR@5 = 0.0146, Coverage@5 = 0.1062, nDCG@5 = 0.017867376002130942, nDCL@5 = 0.0
Test : HR@10 = 0.0449, MRR@10 = 0.0167, Coverage@10 = 0.1486, nDCG@10 = 0.022810376704779294, nDCL@10 = 0.00031328688400001953
Test : HR@20 = 0.0827, MRR@20 = 0.0192, Coverage@20 = 0.2074, nDCG@20 = 0.03182850401950546, nDCL@20 = 0.0007760803573836037
------------------------------------------------------


1137it [00:02, 428.27it/s]


for context 4+5 evaluation:
Test : HR@5 = 0.0246, MRR@5 = 0.0142, Coverage@5 = 0.1059, nDCG@5 = 0.01677526696874801, nDCL@5 = 0.0
Test : HR@10 = 0.0589, MRR@10 = 0.0186, Coverage@10 = 0.1431, nDCG@10 = 0.02712028100059107, nDCL@10 = 0.0005780450156306099
Test : HR@20 = 0.1003, MRR@20 = 0.0215, Coverage@20 = 0.1981, nDCG@20 = 0.03698954841129931, nDCL@20 = 0.0011894372442691652
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Test : HR@5 = 0.0299, MRR@5 = 0.0154, Coverage@5 = 0.1067, nDCG@5 = 0.018939600060349285, nDCL@5 = 0.0
Test : HR@10 = 0.0545, MRR@10 = 0.0185, Coverage@10 = 0.1494, nDCG@10 = 0.026126076323414085, nDCL@10 = 0.000570622876094162
Test : HR@20 = 0.0959, MRR@20 = 0.0213, Coverage@20 = 0.2088, nDCG@20 = 0.035744226202484616, nDCL@20 = 0.0013849021974513384
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [06:31<00:00, 10.88s/it]


Best HR=0.0704 achieved with mlrank=(55, 50, 5)
Best MRR=0.0260 achieved with mlrank=(55, 45, 5)
COV=0.1684 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:03, 377.11it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0229, MRR@5 = 0.0130, Coverage@5 = 0.1230, nDCG@5 = 0.013501595259558496, nDCL@5 = 0.0019002272838202505
Test : HR@10 = 0.0510, MRR@10 = 0.0167, Coverage@10 = 0.1673, nDCG@10 = 0.021426338215084297, nDCL@10 = 0.0030055584084011963
Test : HR@20 = 0.1003, MRR@20 = 0.0200, Coverage@20 = 0.2256, nDCG@20 = 0.03159885631636453, nDCL@20 = 0.005175396057991248
Pipeline ended.


In [ ]:
eucl_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        eucl_matrix[i, j] = abs(i - j) / np.exp(abs(i - j)) if i != j else 1#5 + 1e-2
        
a = np.linalg.cholesky(eucl_matrix)

#for i in range(5):
#    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)

In [ ]:
eucl_matrix

array([[1.        , 0.36787944, 0.27067057, 0.14936121, 0.07326256],
       [0.36787944, 1.        , 0.36787944, 0.27067057, 0.14936121],
       [0.27067057, 0.36787944, 1.        , 0.36787944, 0.27067057],
       [0.14936121, 0.27067057, 0.36787944, 1.        , 0.36787944],
       [0.07326256, 0.14936121, 0.27067057, 0.36787944, 1.        ]])

In [ ]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:02, 449.61it/s]


for context 5 evaluation:
Test : HR@5 = 0.0325, MRR@5 = 0.0173, Coverage@5 = 0.1133, nDCG@5 = 0.02060432236528717, nDCL@5 = 0.00043975373790677223
Test : HR@10 = 0.0484, MRR@10 = 0.0195, Coverage@10 = 0.1615, nDCG@10 = 0.024949103516384515, nDCL@10 = 0.0013205621816481372
Test : HR@20 = 0.0862, MRR@20 = 0.0221, Coverage@20 = 0.2184, nDCG@20 = 0.03389446271350918, nDCL@20 = 0.0019477206355299238
------------------------------------------------------


1137it [00:02, 436.68it/s]


for context 4+5 evaluation:
Test : HR@5 = 0.0273, MRR@5 = 0.0165, Coverage@5 = 0.1067, nDCG@5 = 0.0188046987495604, nDCL@5 = 0.0003402399360022354
Test : HR@10 = 0.0607, MRR@10 = 0.0208, Coverage@10 = 0.1464, nDCG@10 = 0.029122114892130457, nDCL@10 = 0.0006176936534918825
Test : HR@20 = 0.1055, MRR@20 = 0.0239, Coverage@20 = 0.2014, nDCG@20 = 0.03892623484222006, nDCL@20 = 0.002164813185781927
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Test : HR@5 = 0.0317, MRR@5 = 0.0182, Coverage@5 = 0.1087, nDCG@5 = 0.02105086337696857, nDCL@5 = 0.00043975373790677223
Test : HR@10 = 0.0536, MRR@10 = 0.0211, Coverage@10 = 0.1516, nDCG@10 = 0.02782691500401366, nDCL@10 = 0.0007172074553964193
Test : HR@20 = 0.0994, MRR@20 = 0.0242, Coverage@20 = 0.2061, nDCG@20 = 0.03847711975476604, nDCL@20 = 0.0015739320900605947
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [06:32<00:00, 10.89s/it]


Best HR=0.0756 achieved with mlrank=(55, 45, 5)
Best MRR=0.0261 achieved with mlrank=(55, 50, 5)
COV=0.1714 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:03, 371.74it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0264, MRR@5 = 0.0140, Coverage@5 = 0.1285, nDCG@5 = 0.015803032964804557, nDCL@5 = 0.001235387306983765
Test : HR@10 = 0.0563, MRR@10 = 0.0178, Coverage@10 = 0.1714, nDCG@10 = 0.02423695351166098, nDCL@10 = 0.002307115419365183
Test : HR@20 = 0.0959, MRR@20 = 0.0205, Coverage@20 = 0.2325, nDCG@20 = 0.031898621460716925, nDCL@20 = 0.004529182667381669
Pipeline ended.


## Rating distribution attention

In [10]:
rating_dist = []

total_cnt = training.shape[0]

for i in range(5):
    val = training.query(f'rating == {i + 1}').count()[0] / total_cnt
    
    rating_dist.append(val)

rating_dist

[0.056644931644931645,
 0.10500693000693001,
 0.25943898443898444,
 0.3442087192087192,
 0.2347004347004347]

In [ ]:
rat_dist_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        rat_dist_matrix[i, j] = rating_dist[i] / abs(rating_dist[i] - rating_dist[j])
        #rat_dist_matrix[i, j] = diff / np.exp(diff) if i != j else 1. + 1e-1

rat_dist_matrix

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


array([[        inf,  1.17126946,  0.27932245,  0.19698214,  0.31813076],
       [ 2.17126946,         inf,  0.67995553,  0.4389889 ,  0.8096545 ],
       [ 1.27932245,  1.67995553,         inf,  3.06051429, 10.48723499],
       [ 1.19698214,  1.4389889 ,  4.06051429,         inf,  3.14322081],
       [ 1.31813076,  1.8096545 ,  9.48723499,  2.14322081,         inf]])

In [ ]:
rat_dist_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        diff = abs(rating_dist[i] - rating_dist[j])
        rat_dist_matrix[i, j] = diff / np.exp(diff) if i != j else 1. + 1e-1
        
a = np.linalg.cholesky(rat_dist_matrix)

for i in range(5):
    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)
#rat_dist_matrix

In [ ]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:02, 445.40it/s]


for context 5 evaluation:
Test : HR@5 = 0.0237, MRR@5 = 0.0144, Coverage@5 = 0.1117, nDCG@5 = 0.016772111754940346, nDCL@5 = 0.0
Test : HR@10 = 0.0466, MRR@10 = 0.0175, Coverage@10 = 0.1546, nDCG@10 = 0.023598806239983595, nDCL@10 = 0.0005780450156306099
Test : HR@20 = 0.0783, MRR@20 = 0.0197, Coverage@20 = 0.2036, nDCG@20 = 0.03118319086912337, nDCL@20 = 0.0010092849309640722
------------------------------------------------------


1137it [00:02, 421.86it/s]


for context 4+5 evaluation:
Test : HR@5 = 0.0334, MRR@5 = 0.0200, Coverage@5 = 0.1109, nDCG@5 = 0.02287084881401924, nDCL@5 = 0.00043975373790677223
Test : HR@10 = 0.0642, MRR@10 = 0.0240, Coverage@10 = 0.1499, nDCG@10 = 0.03215653999889215, nDCL@10 = 0.001030494339396439
Test : HR@20 = 0.1091, MRR@20 = 0.0270, Coverage@20 = 0.2014, nDCG@20 = 0.042042408086102726, nDCL@20 = 0.002324415387296944
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Test : HR@5 = 0.0334, MRR@5 = 0.0172, Coverage@5 = 0.1144, nDCG@5 = 0.02074601094985069, nDCL@5 = 0.0003787832524831953
Test : HR@10 = 0.0712, MRR@10 = 0.0221, Coverage@10 = 0.1552, nDCG@10 = 0.03205373289389733, nDCL@10 = 0.0012005394879658672
Test : HR@20 = 0.1214, MRR@20 = 0.0255, Coverage@20 = 0.2028, nDCG@20 = 0.04261636016198688, nDCL@20 = 0.0031542602033563837
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [06:30<00:00, 10.84s/it]


Best HR=0.0844 achieved with mlrank=(50, 45, 5)
Best MRR=0.0306 achieved with mlrank=(55, 55, 5)
COV=0.1772 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:03, 344.26it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0317, MRR@5 = 0.0143, Coverage@5 = 0.1348, nDCG@5 = 0.016939382722254145, nDCL@5 = 0.0016141705594669602
Test : HR@10 = 0.0580, MRR@10 = 0.0179, Coverage@10 = 0.1774, nDCG@10 = 0.02469648307522927, nDCL@10 = 0.0024464502508388963
Test : HR@20 = 0.1055, MRR@20 = 0.0211, Coverage@20 = 0.2333, nDCG@20 = 0.034409123222778414, nDCL@20 = 0.004649546849046022
Pipeline ended.


## Trigonometry scale attention

In [ ]:
def rescale_score(x, func=None):
    
    if func is None:
        func = np.arctan
    
    return func(x)

In [ ]:
eucl_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        
        k, l = rescale_score(i + 1), rescale_score(j + 1)
        
        diff = abs(k - l)
        
        eucl_matrix[i, j] = diff / np.exp(diff) if i != j else 5 + 1e-2
        
a = np.linalg.cholesky(eucl_matrix)

for i in range(5):
    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)

In [ ]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1441.94it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0343, MRR@5 == 0.0208, Coverage@5 = 0.1048
Validation : HR@10 = 0.0624, MRR@10 == 0.0244, Coverage@10 = 0.1461
Validation : HR@20 = 0.1038, MRR@20 == 0.0271, Coverage@20 = 0.2014
------------------------------------------------------


1137it [00:01, 1129.38it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0308, MRR@5 == 0.0183, Coverage@5 = 0.1103
Validation : HR@10 = 0.0633, MRR@10 == 0.0228, Coverage@10 = 0.1475
Validation : HR@20 = 0.1108, MRR@20 == 0.0260, Coverage@20 = 0.2003
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0009, MRR@5 == 0.0003, Coverage@5 = 0.1653
Validation : HR@10 = 0.0018, MRR@10 == 0.0004, Coverage@10 = 0.2360
Validation : HR@20 = 0.0026, MRR@20 == 0.0005, Coverage@20 = 0.3249
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:58<00:00,  6.62s/it]


Best HR=0.0844 achieved with mlrank=(50, 50, 5)
Best MRR=0.0303 achieved with mlrank=(50, 50, 5)
COV=0.1750 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1045.84it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0237, MRR@5 == 0.0103, Coverage@5 = 0.1287
Test : HR@10 = 0.0589, MRR@10 == 0.0150, Coverage@10 = 0.1744
Test : HR@20 = 0.0994, MRR@20 == 0.0177, Coverage@20 = 0.2338
Pipeline ended.


In [ ]:
def center_and_rescale_score(x, func=None):
    
    if func is None:
        func = np.arctan
    
    return func(x - 3)

In [ ]:
eucl_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        
        k, l = center_and_rescale_score(i + 1), center_and_rescale_score(j + 1)
        
        diff = abs(k - l)
        
        eucl_matrix[i, j] = 1 / (diff + 1)

similarity = eucl_matrix
        
print(similarity)
    
a = np.linalg.cholesky(similarity)        

attention_matrix = csr_matrix(a)

[[1.         0.75657241 0.47457495 0.34571609 0.31110998]
 [0.75657241 1.         0.56009915 0.38898453 0.34571609]
 [0.47457495 0.56009915 1.         0.56009915 0.47457495]
 [0.34571609 0.38898453 0.56009915 1.         0.75657241]
 [0.31110998 0.34571609 0.47457495 0.75657241 1.        ]]


In [ ]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1482.42it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0290, MRR@5 == 0.0190, Coverage@5 = 0.1111
Validation : HR@10 = 0.0528, MRR@10 == 0.0222, Coverage@10 = 0.1530
Validation : HR@20 = 0.0871, MRR@20 == 0.0245, Coverage@20 = 0.2168
------------------------------------------------------


1137it [00:00, 1424.28it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0281, MRR@5 == 0.0192, Coverage@5 = 0.1103
Validation : HR@10 = 0.0598, MRR@10 == 0.0235, Coverage@10 = 0.1505
Validation : HR@20 = 0.1108, MRR@20 == 0.0270, Coverage@20 = 0.2039
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0299, MRR@5 == 0.0180, Coverage@5 = 0.1109
Validation : HR@10 = 0.0572, MRR@10 == 0.0217, Coverage@10 = 0.1527
Validation : HR@20 = 0.1099, MRR@20 == 0.0253, Coverage@20 = 0.2116
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:38<00:00,  6.07s/it]


Best HR=0.0827 achieved with mlrank=(50, 50, 5)
Best MRR=0.0293 achieved with mlrank=(55, 50, 5)
COV=0.1816 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1105.85it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0246, MRR@5 == 0.0129, Coverage@5 = 0.1312
Test : HR@10 = 0.0554, MRR@10 == 0.0168, Coverage@10 = 0.1816
Test : HR@20 = 0.0985, MRR@20 == 0.0198, Coverage@20 = 0.2454
Pipeline ended.


## Conditional prob attention

In [ ]:
train_new_part = train_new.query('rating == 1| rating == 2')
users = train_new_part.userid.unique()
count12_tot = 0
count12_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 2:
     count12_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count12_loc += 1
  count12_tot += count12_loc

In [ ]:
train_new_part = train_new.query('rating == 1| rating == 3')
users = train_new_part.userid.unique()
count13_tot = 0
count13_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 3:
     count13_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count13_loc += 1
  count13_tot += count13_loc

In [ ]:
train_new_part = train_new.query('rating == 1| rating == 4')
users = train_new_part.userid.unique()
count14_tot = 0
count14_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 4:
     count14_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count14_loc += 1
  count14_tot += count14_loc

In [ ]:
train_new_part = train_new.query('rating == 1| rating == 5')
users = train_new_part.userid.unique()
count15_tot = 0
count15_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count15_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count15_loc += 1
  count15_tot += count15_loc

In [ ]:
train_new_part = train_new.query('rating == 2| rating == 3')
users = train_new_part.userid.unique()
count23_tot = 0
count23_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 3:
     count23_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 2:
      count23_loc += 1
  count23_tot += count23_loc

In [ ]:
train_new_part = train_new.query('rating == 2| rating == 4')
users = train_new_part.userid.unique()
count24_tot = 0
count24_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 4:
     count24_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 2:
      count24_loc += 1
  count24_tot += count24_loc

In [ ]:
train_new_part = train_new.query('rating == 2| rating == 5')
users = train_new_part.userid.unique()
count25_tot = 0
count25_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count25_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 2:
      count25_loc += 1
  count25_tot += count25_loc

In [ ]:
train_new_part = train_new.query('rating == 3| rating == 4')
users = train_new_part.userid.unique()
count34_tot = 0
count34_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 4:
     count34_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 3:
      count34_loc += 1
  count34_tot += count34_loc

In [ ]:
train_new_part = train_new.query('rating == 3| rating == 5')
users = train_new_part.userid.unique()
count35_tot = 0
count35_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count35_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 3:
      count35_loc += 1
  count35_tot += count35_loc

In [ ]:
train_new_part = train_new.query('rating == 4| rating == 5')
users = train_new_part.userid.unique()
count45_tot = 0
count45_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count45_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 4:
      count45_loc += 1
  count45_tot += count45_loc

In [ ]:
train_new_part = train_new.query('rating == 1')
users = train_new_part.userid.unique()
count11_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count11_tot += len(train_new_part_user)-1

In [ ]:
train_new_part = train_new.query('rating == 2')
users = train_new_part.userid.unique()
count22_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count22_tot += len(train_new_part_user)-1

In [ ]:
train_new_part = train_new.query('rating == 3')
users = train_new_part.userid.unique()
count33_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count33_tot += len(train_new_part_user)-1

In [ ]:
train_new_part = train_new.query('rating == 4')
users = train_new_part.userid.unique()
count44_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count44_tot += len(train_new_part_user)-1

In [ ]:
train_new_part = train_new.query('rating == 5')
users = train_new_part.userid.unique()
count55_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count55_tot += len(train_new_part_user)-1

In [ ]:
rat_dist_matrix = np.zeros((5, 5))

rat_dist_matrix[0][0] = count11_tot
rat_dist_matrix[0][1] = rat_dist_matrix[1][0]= count12_tot
rat_dist_matrix[0][2] = rat_dist_matrix[2][0]= count13_tot
rat_dist_matrix[0][3] = rat_dist_matrix[3][0]= count14_tot
rat_dist_matrix[0][4] = rat_dist_matrix[4][0]= count15_tot
rat_dist_matrix[1][1] = count22_tot
rat_dist_matrix[1][2] = rat_dist_matrix[2][1] = count23_tot
rat_dist_matrix[1][3] = rat_dist_matrix[3][1] = count24_tot
rat_dist_matrix[1][4] = rat_dist_matrix[4][1] = count25_tot
rat_dist_matrix[2][2] = count33_tot
rat_dist_matrix[2][3] = rat_dist_matrix[3][2] = count34_tot
rat_dist_matrix[2][4] = rat_dist_matrix[4][2] = count35_tot
rat_dist_matrix[3][3] = count44_tot
rat_dist_matrix[3][4] = rat_dist_matrix[4][3] = count45_tot
rat_dist_matrix[4][4] = count55_tot       

In [ ]:
summ = np.sum(rat_dist_matrix)
rat_dist_matrix /= summ

In [ ]:
rat_dist_matrix[0,0] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[1,1] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[2,2] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[3,3] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[4,4] += np.random.uniform(low=0.0, high=1.0)

In [ ]:
a = np.linalg.cholesky(rat_dist_matrix)

for i in range(5):
    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)

In [ ]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1542.55it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0299, MRR@5 == 0.0164, Coverage@5 = 0.0949
Validation : HR@10 = 0.0554, MRR@10 == 0.0195, Coverage@10 = 0.1337
Validation : HR@20 = 0.0880, MRR@20 == 0.0217, Coverage@20 = 0.1796
------------------------------------------------------


1137it [00:00, 1359.75it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0317, MRR@5 == 0.0188, Coverage@5 = 0.0988
Validation : HR@10 = 0.0598, MRR@10 == 0.0226, Coverage@10 = 0.1309
Validation : HR@20 = 0.0950, MRR@20 == 0.0250, Coverage@20 = 0.1772
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0325, MRR@5 == 0.0203, Coverage@5 = 0.1004
Validation : HR@10 = 0.0624, MRR@10 == 0.0244, Coverage@10 = 0.1304
Validation : HR@20 = 0.1029, MRR@20 == 0.0272, Coverage@20 = 0.1769
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:55<00:00,  6.54s/it]


Best HR=0.0800 achieved with mlrank=(45, 45, 5)
Best MRR=0.0279 achieved with mlrank=(40, 45, 5)
COV=0.1560 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1100.26it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0290, MRR@5 == 0.0126, Coverage@5 = 0.1161
Test : HR@10 = 0.0528, MRR@10 == 0.0157, Coverage@10 = 0.1549
Test : HR@20 = 0.1038, MRR@20 == 0.0190, Coverage@20 = 0.2077
Pipeline ended.
